<img src='./img/LogoWekeo_Copernicus_RGB_0.png' align='right' width='20%'></img>

# Tutorial on basic land applications (data download)

In this tutorial we will use the WEkEO Jupyterhub to access and analyse data from the Copernicus Sentinel-2 and products from the [Copernicus Land Monitoring Service (CLMS)](https://land.copernicus.eu/). 
A region in northern Corsica has been selected as it contains representative landscape features and process elements which can be used to demonstrate the capabilities and strengths of Copernicus space component and services.
The tutorial comprises the following steps:
1. [Search and download data](#search_download): We will select and download a Sentinel-2 scene and the CLMS CORINE Land Cover (CLC) data from their original archive locations via WEkEO using the Harmonised Data Access (HAD) API.
2.	Read and view Sentinel-2 data: Once downloaded, we will read and view the Sentinel-2 data in geographic coordinates as true colour image.
3.	Process and view Sentinel-2 data as a vegetation index: We will see how the vegetation density and health can be assessed from optical EO data to support crop and landscape management practices.
4.	Read and view the CLC data: Display the thematic CLC data with the correct legend.
5.	CLC2018 burnt area in the Sentinel-2 NDVI data: The two products give different results, but they can be combined to provide more information.

<img src='./img/Intro_banner.jpg' align='center' width='100%'></img>

## <a id='search_download'></a>1. Search and download data

Before we begin we must prepare our environment. This includes importing the various python libraries that we will need.

### Load required libraries

In [1]:
import os
import sys
import json
import time
import base64
from IPython.core.display import HTML

import requests
import warnings
warnings.filterwarnings('ignore')
import zipfile

### Load helper functions

In [2]:
from hda_api_functions import *

### Search for the dataset ID from the WEkEO landing platform

The [WEkEO land platform](https://wekeo.eu/) offers access to a huge amount of data. Under [WEkEO DATA](https://wekeo.eu/data?view=catalogue), clicking the + to add a layer, opens a catalogue search. Here you can use free text, or you can use the filter options on the left to refine your search and look up data by satellite platform, sensor, Copernicus service, area (region of interest), general time period (past or future), as well as through a variety of flags.

<img src='./img/WEkEO_data_01.jpg' align='middle' width='50%'></img>

You can click on the datasets you are interested in and you will be guided to a range of details including the dataset temporal and spatial extent, collection ID, and metadata.

When searching for the products of Sentinel-2, click under Platform in the Filters on the left-hand side of the catalogue panel. Two datasets are available, but we will use “SENTINEL-2 Level-1C”. Once you have found it, select 'Details' to read the dataset description.
The dataset description provides the following information:
* Abstract, containing a general description of the dataset,
* Classification, including the Dataset ID
* Resources, such as a link to the Product Data Format Specification guide, and JSON metadata
* Contacts, where you can find further information about the data source from its provider.
* Raw metadata, details of the dataset in XML format.

<img src='./img/WEkEO_data_02.jpg' align='centre' width='50%'></img>

You need the information from here to request data from the Harmonised Data Access API. 

This process is explained in a previous training session which can be found at the [WEkEO YouTube Channel](https://www.youtube.com/channel/UCvS3VvKmMKs1M2ZkmQPyRlw). The YouTube channel also contains lots of other useful training and support information, such as how to [clone the GitHub to refresh the training materials](https://www.youtube.com/watch?v=pmCkvXcnZxY&list=PLAT-b7DuvMgogqJa5_ii5GteOYmXCce24&index=2).

For this session the details of the required datasets have already been prepared as JSON files which are used below.


In [3]:
dataset_id_S2 = "EO:EO:ESA:DAT:SENTINEL-2:MSI"
dataset_id_corine = "EO:CLMS:DAT:CORINE"

filename_json_S2 = os.path.join(os.getcwd(),'S2_request.json') 
filename_json_corine = os.path.join(os.getcwd(), 'corine_corsica.json')

### Generate the WEkEO API keys

To interact with WEkEO's Harmonised Data Access (HDA) API, each user requires an API token. This token can be generated from an API Key. Your current token can be found in your [WEkEO Dashboard](https://www.wekeo.eu/web/guest/dashboard) under Settings, but it is better to do it in a programmatic way, as shown below, as tokens have a limited lifetime of 1 hour!

In [4]:
# your WEkEO API username and password (needs to be in '  ')
user_name = str(os.environ.get('WEKEO_USERNAME'))
password = str(os.environ.get('WEKEO_PASS'))

In [5]:
api_key = generate_api_key(user_name, password)
display(HTML('Your API key is: <b>'+api_key+'</b>'))

### Initialise the Harmonised Data Access (HDA) API request
To initialise an API request, you have to initialise a dictionary that contains information on dataset_id, api_key and download_directory_path.

Please enter the path of the directory where the data shall be downloaded to.

In [6]:
# Enter here the directory path where you want to download the data to

download_dir_path = os.path.join(os.getcwd(),'data/from_wekeo')

print(download_dir_path)

/home/develop/Workspace/WEkEO-notebooks/WEkEO-training_session-Land-Themes-JN/Geoff/wekeo-jupyter-lab/land/data/from_wekeo


In [7]:
hda_dict_S2 = init(dataset_id_S2, api_key, download_dir_path)
hda_dict_corine = init(dataset_id_corine, api_key, download_dir_path)

### Request access token
Once initialised, you can request an access token with the function get_access_token. The access token is stored in the hda_dict dictionary.


In [8]:
hda_dict_S2 = get_access_token(hda_dict_S2)
hda_dict_corine = get_access_token(hda_dict_corine)

Getting an access token. This token is valid for one hour only.
Success: Access token is c2bd023e-dec3-32b8-b417-7792265d1736
Getting an access token. This token is valid for one hour only.
Success: Access token is c2bd023e-dec3-32b8-b417-7792265d1736


### Accept Terms and Conditions (if applicable)
You might need to accept the Terms and Conditions, which you can do with the function acceptTandC.


In [9]:
hda_dict_S2 = acceptTandC(hda_dict_S2)
hda_dict_corine = acceptTandC(hda_dict_corine)

Copernicus_General_License Terms and Conditions already accepted
Copernicus_General_License Terms and Conditions already accepted


### Load data descriptor file and request data
The Harmonised Data Access API can read your data request from a JSON file. In this JSON-based file, you can describe the dataset you are interested in downloading. The file is in principle a dictionary. Following keys can be defined:

- datasetID: the dataset's collection ID
- stringChoiceValues: type of dataset, e.g. 'Non Time Critical'
- dataRangeSelectValues: time period you would like to retrieve data
- boundingBoxValues: optional to define a subset of a global field

You can also get a specific example of a JSON file for a particular query from the WEkEO DATA portal.

### Displaying a JSON query from a request made to the Harmonised Data Access API through the data portal

You can load the JSON file with json.load(). Alternatively, you can copy paste the dictionary describing your data into a cell as shown in the YouTube video.

For this training session JSON file has already been prepared to select the correct Sentinel-2 scene and CLC data to correspond with later tasks. The details were loaded earlier in the JN. The following cell reads that JSON file and displays the contents.

In [10]:
try:
    with open(filename_json_S2, 'r') as f:
        data_S2 = json.load(f)
    print('Your JSON file:')
    print(data_S2)
except:
    print('Your JSON file is not in the correct format, or is not found, please check it!')

Your JSON file:
{'datasetId': 'EO:ESA:DAT:SENTINEL-2:MSI', 'boundingBoxValues': [{'name': 'bbox', 'bbox': [9.425764239078317, 42.74275713340862, 9.735642520957134, 43.05969192516483]}], 'dateRangeSelectValues': [{'name': 'position', 'start': '2017-08-02T00:00:00.000Z', 'end': '2017-08-02T23:00:00.000Z'}], 'stringChoiceValues': [{'name': 'processingLevel', 'value': 'LEVEL2A'}]}


In [11]:
try:
    with open(filename_json_corine, 'r') as f:
        data_corine = json.load(f)
    print('Your JSON file:')
    print(data_corine)
except:
    print('Your JSON file is not in the correct format, or is not found, please check it!')

Your JSON file:
{'datasetId': 'EO:CLMS:DAT:CORINE', 'stringChoiceValues': [{'name': 'product_type', 'value': 'Corine Land Cover 2018'}, {'name': 'format', 'value': 'GeoTiff100mt'}]}


### Initiating the request by setting a job ID

In [12]:
hda_dict_S2 = get_job_id(hda_dict_S2, data_S2)
hda_dict_corine = get_job_id(hda_dict_corine, data_corine)


Query successfully submitted. Job ID is mX76sM3GtOq35jqc_mkb7bswE-E
Query successfully submitted. Status is running
Query successfully submitted. Status is running
Query successfully submitted. Status is running
Query successfully submitted. Status is running
Query successfully submitted. Status is completed
Query successfully submitted. Job ID is nGxnVcbbqBWLDbdQoEldY9QMTAU
Query successfully submitted. Status is running
Query successfully submitted. Status is running
Query successfully submitted. Status is running
Query successfully submitted. Status is completed


### Build list of file names to be ordered and downloaded

In [13]:
hda_dict_S2 = get_results_list(hda_dict_S2)
hda_dict_corine = get_results_list(hda_dict_corine)


************** Results *******************************
{
    "content": [
        {
            "downloadUri": null,
            "filename": "S2A_MSIL2A_20170802T101031_N0205_R022_T32TNN_20170802T101051.SAFE",
            "order": null,
            "productInfo": {
                "datasetId": "EO:ESA:DAT:SENTINEL-2:MSI",
                "product": "S2A_MSIL2A_20170802T101031_N0205_R022_T32TNN_20170802T101051.SAFE",
                "productEndDate": "2017-08-02T10:10:31.026000Z",
                "productStartDate": "2017-08-02T10:10:31.026000Z"
            },
            "size": 845615368,
            "url": "d5748e25-b24b-58c3-ae91-ed6a50460d9a/S2A_MSIL2A_20170802T101031_N0205_R022_T32TNN_20170802T101051.SAFE"
        }
    ],
    "itemsInPage": 1,
    "nextPage": null,
    "page": 0,
    "pages": 1,
    "previousPage": null,
    "totItems": 1
}
*******************************************
************** Results *******************************
{
    "content": [
        {
            "

### Create an order ID for each file to be downloaded

In [14]:
hda_dict_S2 = get_order_ids(hda_dict_S2)
hda_dict_corine = get_order_ids(hda_dict_corine)

Query successfully submitted. Order ID is W5mSZ9_UXR6Qas15ybV3QyQdSJ4
Query successfully submitted. Status is completed
Query successfully submitted. Order ID is wVKR4dc9wRvl70gpVU6GbfZ4SHM
Query successfully submitted. Status is running
Query successfully submitted. Status is completed


### Download requested data

In [15]:
hda_dict_S2 = download_data(hda_dict_S2)
hda_dict_corine = download_data(hda_dict_corine)

IsADirectoryError: [Errno 21] Is a directory: '/home/develop/Workspace/WEkEO-notebooks/WEkEO-training_session-Land-Themes-JN/Geoff/wekeo-jupyter-lab/land/data/from_wekeo/S2A_MSIL2A_20170802T101031_N0205_R022_T32TNN_20170802T101051.SAFE'

### Decompressing Sentinel 2 and Corine Land Cover data

In [16]:
#rename S2 file
for item in os.listdir(download_dir_path): 
    if item.endswith('SAFE'):
        os.rename(os.path.join(download_dir_path, item), os.path.join(download_dir_path, item +".zip"))

In case you have permission issues when renaming the .SAFE file with the code above, please manually do that by right-clicking the file, then rename, then add .zip to the end of the filename.

For example:
- From: data/from_wekeo/S2A_MSIL2A_20170802T101031_N0205_R022_T32TNN_20170802T101051.SAFE   
- To: data/from_wekeo/S2A_MSIL2A_20170802T101031_N0205_R022_T32TNN_20170802T101051.SAFE.zip

In [17]:
extension = ".zip"
for item in os.listdir(download_dir_path): # loop through items in dir
    print(item)
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.join(download_dir_path, item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(download_dir_path) # extract file to dir
        zip_ref.close() # close file

.gitkeep
u2018_clc2018_v2020_20u1_raster100m.zip
S2A_MSIL2A_20170802T101031_N0205_R022_T32TNN_20170802T101051.SAFE.zip
.ipynb_checkpoints


<hr>

<p><img src='./img/all_partners_wekeo_2.png' align='left' alt='Logo EU Copernicus' width='100%'></img></p>